In [1]:
import wandb

wandb.login(key='b123af3ff1bc7e54569d0976c6405a5b3b6d2902')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import re

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
def preprocess_text(text):
    """
    Preprocess text to retain only alphabetic characters and convert to lowercase.
    
    Parameters:
        text (str): Input string.
        
    Returns:
        str: Preprocessed string.
    """
    text = re.sub(r"\s+", " ", text).strip()
    text = text.lower()
    return text


def all_preprocessing(df):
    df['text'] = df['text'].apply(preprocess_text)
    return df

def extract_words_from_mask(df, text_col='text', mask_col='label_new'):
    """
    Extracts substrings from `text` based on the boolean mask in `label_new`.
    
    Parameters:
        df (pd.DataFrame): Input dataframe with columns `text` and `label_new`.
        text_col (str): Name of the column containing the text.
        mask_col (str): Name of the column containing the boolean mask.
        
    Returns:
        pd.Series: A Series where each entry is a list of words extracted from `text`.
    """
    def process_row(row):
        text = row[text_col]
        mask = row[mask_col]
        words = []
        current_word = []
        
        for char, include in zip(text, mask):
            if include:
                current_word.append(char)
            elif current_word:  # If a word is in progress and we hit a `0`
                words.append("".join(current_word))
                current_word = []  # Reset for the next word
        
        if current_word:  # Append the last word if still in progress
            words.append("".join(current_word))
        words = sorted(words)
        return ",".join(words)
    
    return df.apply(process_row, axis=1)


def to_out_labels(texts, labels):
    def process_row(i):
        text = texts[i]
        mask = labels[i]
        words = []
        current_word = []
        
        for char, include in zip(text, mask):
            if include:
                current_word.append(char)
            elif current_word:  # If a word is in progress and we hit a `0`
                words.append("".join(current_word))
                current_word = []  # Reset for the next word
        
        if current_word:  # Append the last word if still in progress
            words.append("".join(current_word))

        words = sorted(words)
        return ",".join(words)
    out_labels = [process_row(i) for i in range(len(texts))]
    return out_labels

In [4]:
import ast

df = pd.read_csv('/kaggle/input/wb-contest-2/wb_contest_2_new_dataset.csv', index_col='ID')
df['label'] = df['label'].apply(lambda x: ast.literal_eval(x))

df.tail()

,text,label
ID,,
248088,мне ее порвали суки,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
248089,"полное дерьмо, удалите этот товар и заблокируй...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, ..."
248090,херня. деньги на ветер.,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
248091,"это вообще что , за 💩 гов... ще?? темнотища уж...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
248092,"не берите!!!!! мелкие, порезанные, подпорченны...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
# Custom Tokenizer: Split into characters and map each character to an index
class CharTokenizer:
    def __init__(self, texts):
        # Create a vocabulary from the unique characters in the dataset + a padding token (0)
        self.vocab = {char: idx + 1 for idx, char in enumerate(set(''.join(texts)))}  # index 0 is for padding
        self.vocab_size = len(self.vocab) + 1  # Add 1 for padding
        self.vocab['<PAD>'] = 0  # Adding padding token
        self.reverse_vocab = {v: k for k, v in self.vocab.items()}

    def encode(self, text):
        # Return the indices of each character in the text based on the vocabulary
        return [self.vocab.get(char, self.vocab['<PAD>']) for char in text]

    def decode(self, token_ids):
        # Decode token IDs back to characters
        return ''.join([self.reverse_vocab.get(idx, '<PAD>') for idx in token_ids])

# Custom Dataset
class TokenClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=1024):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize the text into character-level tokens
        input_ids = self.tokenizer.encode(text)
        
        # Pad the sequence to max_len if necessary
        input_ids = input_ids + [0] * (self.max_len - len(input_ids)) if len(input_ids) < self.max_len else input_ids[:self.max_len]
        
        # Pad the labels to max_len if necessary
        labels_padded = label + [0] * (self.max_len - len(label)) if len(label) < self.max_len else label[:self.max_len]
        #print(sum(labels_padded))
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor([1] * len(input_ids) + [0] * (self.max_len - len(input_ids)), dtype=torch.long),  # Attention mask
            'labels': torch.tensor(labels_padded, dtype=torch.long)
        }

class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=1024):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        
        input_ids = self.tokenizer.encode(text)
        input_ids = input_ids + [0] * (self.max_len - len(input_ids)) if len(input_ids) < self.max_len else input_ids[:self.max_len]

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor([1] * len(input_ids) + [0] * (self.max_len - len(input_ids)), dtype=torch.long)  # Attention mask
        }

In [6]:
# Define the model for token classification (character-level)
class CharTokenClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_dim=128, num_classes=2, num_layers=2, bidirectional=True, dropout=0.3):
        super(CharTokenClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)  # Character embeddings
        self.lstm = nn.LSTM(embedding_dim, lstm_dim, num_layers=num_layers, batch_first=True, bidirectional=bidirectional, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(lstm_dim * 2 if bidirectional else lstm_dim, num_classes)  # Adjusted for bidirectional

    def forward(self, input_ids, attention_mask):
        # Get embeddings for each character token
        x = self.embedding(input_ids)  # (batch_size, seq_len, embedding_dim)
        x, (hn, cn) = self.lstm(x)  # (batch_size, seq_len, hidden_dim)
        x = self.dropout(x)
        logits = self.classifier(x)  # (batch_size, seq_len, num_classes)
        return logits

In [7]:
texts = df['text'].values  # or df['text'].to_numpy()
labels = df['label'].values  # or df['label'].to_numpy()

tokenizer = CharTokenizer(texts)
print(f"Vocabulary size: {tokenizer.vocab_size}")

# Create the dataset and dataloader
dataset = TokenClassificationDataset(texts, labels, tokenizer)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

Vocabulary size: 1131


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function, and optimizer
model = CharTokenClassifier(vocab_size=tokenizer.vocab_size, embedding_dim=32)
loss_fn = nn.CrossEntropyLoss()  # For token classification (binary)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch"):
        # Get inputs and labels
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(input_ids, attention_mask)
        
        # Compute the loss
        loss = loss_fn(logits.view(-1, 2), labels.view(-1))  # Flatten the logits and labels
        total_loss += loss.item()
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


Epoch 1/5: 100%|██████████| 957/957 [06:47<00:00,  2.35batch/s]


Epoch 1/5, Loss: 0.0055


Epoch 2/5: 100%|██████████| 957/957 [06:47<00:00,  2.35batch/s]


Epoch 2/5, Loss: 0.0008


Epoch 3/5: 100%|██████████| 957/957 [06:47<00:00,  2.35batch/s]


Epoch 3/5, Loss: 0.0005


Epoch 4/5: 100%|██████████| 957/957 [06:47<00:00,  2.35batch/s]


Epoch 4/5, Loss: 0.0004


Epoch 5/5: 100%|██████████| 957/957 [06:46<00:00,  2.35batch/s]

Epoch 5/5, Loss: 0.0003


In [9]:
torch.save(model.state_dict(), '/kaggle/working/model.pth')

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function, and optimizer
model = CharTokenClassifier(vocab_size=tokenizer.vocab_size, embedding_dim=32)
model.load_state_dict(torch.load('/kaggle/working/model.pth', weights_only=True))
model.to(device)
model.eval()  # Set the model to evaluation mode

# Function to run inference on the entire dataset or a sample
def run_eval(dataloader):
    all_preds = []
    all_labels = []

    with torch.no_grad():  # Disable gradient computation for inference
        for batch in tqdm(dataloader, desc="Running Inference"):
            # Move batch data to the correct device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Run the model on the batch
            logits = model(input_ids, attention_mask)  # (batch_size, seq_len, num_classes)

            # Apply softmax to get class probabilities
            probs = torch.softmax(logits, dim=-1)  # Shape: (batch_size, seq_len, num_classes)

            # Get predicted class for each token
            predicted_class = torch.argmax(probs, dim=-1)  # Shape: (batch_size, seq_len)

            # Store predictions and labels
            for i in range(len(labels)):
                all_preds.append(predicted_class[i].cpu().numpy())  # Move to CPU for easy handling
                all_labels.append(labels[i].cpu().numpy())

    return all_preds, all_labels

# Run inference
predictions, ground_truth = run_eval(dataloader)

Running Inference: 100%|██████████| 957/957 [03:53<00:00,  4.09it/s]


In [11]:
i = 244736
text = dataset.texts[i]
# Example: Print the first prediction and corresponding ground truth
print(f"Text: {text}")
print(f"Prediction: {predictions[i]}")
print(f"Ground truth: {ground_truth[i]}")

predicted = pd.DataFrame({"text": dataset.texts, "prediction": predictions, "label": ground_truth})
predicted['label_out'] = extract_words_from_mask(predicted, mask_col='prediction')

predicted

Text: херня. деньги на ветер.
Prediction: [0 0 0 ... 0 0 0]
Ground truth: [0 0 0 ... 0 0 0]


,text,prediction,label,label_out
0,"брюки отличные, качественные, но к сожалению к...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",брюки
1,"отличный аппарат, в комплекте кабель и работет...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
2,супер 👍 спасибо большое,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
3,получил быстро данные наушники! к наушникам пр...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
4,всё дошло в целости и сохранности),"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
...,...,...,...,...
244734,мне ее порвали суки,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
244735,"полное дерьмо, удалите этот товар и заблокируй...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
244736,херня. деньги на ветер.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
244737,"это вообще что , за 💩 гов... ще?? темнотища уж...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",


In [12]:
predicted.to_csv("/kaggle/working/predicted.csv")

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function, and optimizer
model = CharTokenClassifier(vocab_size=tokenizer.vocab_size, embedding_dim=32)
model.load_state_dict(torch.load('/kaggle/working/model.pth', weights_only=True))
model.to(device)
model.eval()

CharTokenClassifier(
  (embedding): Embedding(1131, 32)
  (lstm): LSTM(32, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=256, out_features=2, bias=True)
)

In [14]:
df_test = pd.read_csv('/kaggle/input/wildberries-winter-school-24-contest-detected/test.csv')
df_test = all_preprocessing(df_test)

texts = df_test['text'].values  # or df['text'].to_numpy()

In [15]:
dataset_test = InferenceDataset(texts, tokenizer)
dataloader_test = DataLoader(dataset_test, batch_size=256, shuffle=False)

def run_inference(model, dataloader, device):
    all_preds = []

    with torch.no_grad():  # Disable gradient computation for inference
        for batch in tqdm(dataloader, desc="Running Inference"):
            # Move batch data to the correct device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Run the model on the batch
            logits = model(input_ids, attention_mask)  # (batch_size, seq_len, num_classes)

            # Apply softmax to get class probabilities
            probs = torch.softmax(logits, dim=-1)  # Shape: (batch_size, seq_len, num_classes)

            # Get predicted class for each token
            predicted_class = torch.argmax(probs, dim=-1)  # Shape: (batch_size, seq_len)

            # Store predictions and labels
            for i in range(len(predicted_class)):
                all_preds.append(predicted_class[i].cpu().numpy())  # Move to CPU for easy handling

    return all_preds

# Run inference
predictions = run_inference(model, dataloader_test, device)

df_test["label_new"] = predictions
df_test.to_csv('/kaggle/working/out.csv')

Running Inference: 100%|██████████| 262/262 [00:54<00:00,  4.80it/s]


In [16]:
df_test['label'] = extract_words_from_mask(df_test)

df_test.to_csv('/kaggle/working/out_with_words.csv')
df_test

,ID,text,label_new,label
0,0,"хороший, подошкл","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
1,1,"совсем тонюсенький саженец, не досмотрела в оп...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
2,2,"когтеточка хорошая, но вот ткань на основании ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
3,3,"много затяжек, не порадовала покупка","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
4,4,рекомендую 💣,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",
...,...,...,...,...
66944,71995,пачка как пачка а внутри совсем другое некторв...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",говно
66945,71996,"отвратительное качество!!! через год тряпка, у...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",г…
66946,71997,"вес 100гр, не понимаю откуда хорошие отзывы , ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",г...
66947,71998,"приобрел и установил радиатор год назад , авто...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",говно


In [17]:
df_test.drop(columns=['label_new', 'text']).set_index("ID").to_csv("/kaggle/working/final.csv")